In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import matplotlib
#matplotlib.use('agg')
matplotlib.style.use('ggplot')
import pickle as pkl
from matplotlib import pyplot as plt
from collections import Counter
from functools import reduce
import random
random.seed(1991)
import glob

In [2]:
novel_compounds_list = pkl.load( open( "/data/dharp/compounding/datasets/novel_compounds_list.pkl", "rb" ) )
m, h = zip(*novel_compounds_list)
heads_list=list(set(h))
modifiers_list=list(set(m))

In [3]:
constituents=pd.read_pickle("/data/dharp/compounding/datasets/constituents_CompoundAgnostic_DecadeAgnostic_300.pkl")
#constituents.index.names=['constituent','decade']
#constituents.reset_index(inplace=True)
constituents.info()
#constituents=constituents.drop(['decade'],axis=1).groupby(['constituent']).mean()
constituents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 76597 entries, a_n to x_n ----_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 175.9+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 76597 entries, a_n to x_n ----_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 175.9+ MB


In [4]:
modifiers=constituents.loc[constituents.index.isin(modifiers_list)]
modifiers.index.names=['modifier']
modifiers.info()
modifiers.head()

<class 'pandas.core.frame.DataFrame'>
Index: 7901 entries, a_n to zuni_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 18.1+ MB


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
modifier,,,,,,,,,,,,,,,,,,,,,
a_n,0.947,-0.216,-0.079,0.098,0.031,-0.129,-0.041,-0.053,0.022,0.115,...,-0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000
aaa_n,0.538,-0.114,0.006,0.054,-0.015,-0.046,0.012,0.015,-0.009,0.024,...,0.019,0.011,0.117,0.047,0.024,0.016,0.057,-0.078,0.033,-0.003
aarhus_n,0.032,0.051,0.015,-0.058,0.106,0.018,0.003,-0.011,0.008,-0.018,...,-0.016,0.006,0.009,-0.007,0.013,0.013,0.003,0.014,0.015,-0.001
abbreviated_n,0.709,-0.158,-0.037,-0.021,0.008,-0.138,0.047,0.026,-0.034,-0.090,...,-0.005,-0.003,0.061,-0.018,-0.000,0.072,0.027,-0.043,-0.005,-0.025
abduction_n,0.511,-0.075,-0.035,-0.049,0.049,-0.010,-0.033,-0.007,0.046,-0.029,...,-0.099,0.071,0.035,0.001,0.045,-0.092,0.032,0.035,0.044,-0.061


In [5]:
heads=constituents.loc[constituents.index.isin(heads_list)]
heads.index.names=['head']
heads.info()
heads.head()

<class 'pandas.core.frame.DataFrame'>
Index: 7619 entries, a_n to zwingli_n
Columns: 300 entries, 0 to 299
dtypes: float64(300)
memory usage: 17.5+ MB


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
head,,,,,,,,,,,,,,,,,,,,,
a_n,0.947,-0.216,-0.079,0.098,0.031,-0.129,-0.041,-0.053,0.022,0.115,...,-0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000
aaron_n,0.632,-0.158,-0.029,0.063,-0.004,-0.006,0.038,0.057,-0.007,-0.009,...,-0.030,0.066,-0.032,-0.067,-0.080,-0.042,-0.027,-0.032,0.005,-0.064
ab_n,0.815,-0.224,-0.051,0.066,0.005,-0.142,0.056,0.048,-0.043,-0.136,...,0.040,0.016,0.011,0.025,-0.023,-0.006,-0.028,-0.032,-0.053,-0.058
abandonment_n,0.871,-0.150,-0.024,0.037,0.000,-0.025,0.029,0.028,-0.010,-0.068,...,-0.044,-0.009,0.037,0.017,0.006,0.012,0.019,0.001,-0.048,-0.024
abbreviation_n,0.679,-0.186,-0.050,0.102,-0.031,-0.105,0.054,0.037,-0.035,-0.070,...,0.017,0.007,-0.049,0.007,-0.005,0.013,-0.003,-0.015,0.008,0.009


In [6]:
novel_compounds=pd.DataFrame(novel_compounds_list)
novel_compounds.columns=['modifier','head']
novel_compounds

,modifier,head
0,a_n,peaceful_n
1,a_n,robinson_n
2,a_n,supply_n
3,india_n,peaceful_n
4,india_n,standard_n
5,india_n,award_n
6,india_n,joint_n
7,india_n,companion_n
8,india_n,foundation_n
9,india_n,gentleman_n


In [7]:
positive_df=pd.merge(novel_compounds,heads.reset_index(),on=["head"])
positive_df=pd.merge(positive_df,modifiers.reset_index(),on=["modifier"])
positive_df['Plausibility']=True
positive_df

,modifier,head,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,...,291_y,292_y,293_y,294_y,295_y,296_y,297_y,298_y,299_y,Plausibility
0,a_n,peaceful_n,0.462,0.058,-0.035,-0.137,0.018,0.010,-0.012,-0.004,...,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,True
1,a_n,robinson_n,0.452,-0.092,-0.024,0.025,0.017,-0.005,0.011,0.013,...,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,True
2,a_n,supply_n,0.572,-0.120,-0.016,0.011,0.074,-0.038,-0.014,0.023,...,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,True
3,india_n,peaceful_n,0.462,0.058,-0.035,-0.137,0.018,0.010,-0.012,-0.004,...,-0.033,0.028,0.002,-0.037,-0.052,-0.008,-0.064,0.010,-0.016,True
4,india_n,standard_n,0.574,-0.136,-0.026,0.064,0.054,-0.024,0.010,0.040,...,-0.033,0.028,0.002,-0.037,-0.052,-0.008,-0.064,0.010,-0.016,True
5,india_n,award_n,0.857,-0.228,-0.043,0.114,-0.007,-0.109,0.036,0.025,...,-0.033,0.028,0.002,-0.037,-0.052,-0.008,-0.064,0.010,-0.016,True
6,india_n,joint_n,0.509,-0.082,-0.015,0.014,0.010,-0.044,0.027,0.020,...,-0.033,0.028,0.002,-0.037,-0.052,-0.008,-0.064,0.010,-0.016,True
7,india_n,companion_n,0.913,-0.285,-0.059,0.192,-0.042,-0.054,0.030,0.013,...,-0.033,0.028,0.002,-0.037,-0.052,-0.008,-0.064,0.010,-0.016,True
8,india_n,foundation_n,0.716,-0.162,-0.037,0.091,0.022,-0.048,0.039,0.041,...,-0.033,0.028,0.002,-0.037,-0.052,-0.008,-0.064,0.010,-0.016,True
9,india_n,gentleman_n,0.848,-0.270,-0.051,0.222,-0.050,0.017,0.028,0.008,...,-0.033,0.028,0.002,-0.037,-0.052,-0.008,-0.064,0.010,-0.016,True


In [8]:
def neg_df_creator(file):
    pkl_file=pkl.load( open(file,'rb'))
    df=pd.DataFrame(pkl_file)
    df.columns=['modifier','head']
    negative_df=pd.merge(df,heads.reset_index(),on=["head"])
    negative_df=pd.merge(negative_df,modifiers.reset_index(),on=["modifier"])
    negative_df['Plausibility']=False
    return negative_df

In [9]:
def df_joiner(files):
    df_list=[]
    for file in files:
        neg_df=neg_df_creator(file)
        whole_df=pd.concat([neg_df,positive_df])
        df_list.append(whole_df)
    
    return df_list

In [10]:
corrupt_modifier_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_modifier*"):
    corrupt_modifier_files.append(file)
corrupt_modifiers=df_joiner(corrupt_modifier_files)

In [11]:
corrupt_head_files=[]
for file in glob.glob("/data/dharp/compounding/datasets/corrupt_head*"):
    corrupt_head_files.append(file)
corrupt_heads=df_joiner(corrupt_head_files)

In [14]:
XGBClassifier()

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='binary:logistic',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)

In [15]:
acc_ch=[]
for i,corrupt_head in enumerate(corrupt_heads):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
    # fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
    # make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    # evaluate predictions
    print("DF",i+1)
    accuracy = accuracy_score(y_test, predictions)
    acc_ch.append(accuracy)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))


DF 1
Accuracy: 66.31%
DF 2
Accuracy: 66.98%
DF 3
Accuracy: 68.06%
DF 4
Accuracy: 67.86%
DF 5
Accuracy: 68.25%
DF 6
Accuracy: 67.81%
DF 7
Accuracy: 67.84%
DF 8
Accuracy: 67.17%
DF 9
Accuracy: 67.88%
DF 10
Accuracy: 66.99%


In [16]:
round(np.mean(acc_ch)*100,2)

67.52

In [17]:
round(np.std(acc_ch)*100,2)

0.58

In [18]:
acc_cm=[]
for i,corrupt_head in enumerate(corrupt_modifiers):
    data=corrupt_head.drop(['modifier','head'],axis=1)
    X, Y = data.iloc[:,:-1],data.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1991)
# fit model no training data
    model = XGBClassifier(njobs=-1)
    model.fit(X_train, y_train)
# make predictions for test data
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
# evaluate predictions
    accuracy = accuracy_score(y_test, predictions)
    acc_cm.append(accuracy)
    print("DF",i+1)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))

DF 1
Accuracy: 64.08%
DF 2
Accuracy: 64.42%
DF 3
Accuracy: 65.04%
DF 4
Accuracy: 64.49%
DF 5
Accuracy: 64.31%
DF 6
Accuracy: 64.82%
DF 7
Accuracy: 64.22%
DF 8
Accuracy: 63.68%
DF 9
Accuracy: 64.60%
DF 10
Accuracy: 64.25%


In [19]:
round(np.mean(acc_cm)*100,2)

64.39

In [20]:
round(np.std(acc_cm)*100,2)

0.36